# After sales text clustering using Doc2Vec
## Adding CODART and Characteristics from A3ERP and improving the text preprocessing

## Data preprocessing (Merging the translated text)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from datetime import date
import os
pd.options.mode.chained_assignment = None


today_date = date.today().isoformat()
data_version = '2024-05-14'
data_base_path = f"../DATA/processed/{data_version}"
model_base_path = f"../MODELS/{today_date}"


In [2]:
def query_data(query):
    """
    Function to query data from the database using sqlalchemy
    :param query: 
    :return: pd.DataFrame
    
    Connection parameters:
    user = readmyzone
    password = (get from environment variable MYSQL_PASSWORD)
    host = 192.168.2.7
    port = 3306
    """
    
    # Create the connection string
    user = 'readmyzone'
    password = os.environ.get('MYSQL_PASSWORD')
    host = '192.168.2.7'
    port = '3306'
    db = 'myzone'
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{db}'
    
    # Create the engine
    engine = create_engine(connection_string)
    
    try:
        # Query the data
        data = pd.read_sql(query, engine)
    except Exception as e:
        print(e)
        data = None
    
    return data

## Load the data

In [3]:
sav_incidencias = query_data('SELECT * FROM sav_incidencias')
sav_piezas = query_data('SELECT * FROM sav_piezas')
sav_estados = query_data('SELECT * FROM sav_estados')
sav_incidencias_tipo = query_data('SELECT * FROM sav_incidencias_tipo')

In [4]:
dataset = sav_incidencias.merge(sav_piezas, left_on='codigo', right_on='codigo_incidencia', how='left', suffixes=(None, '_pieza'))
dataset = dataset.merge(sav_estados, left_on='estado', right_on='id', how='left', suffixes=(None, '_estado'))
dataset = dataset.merge(sav_incidencias_tipo, left_on='tipo', right_on='id', how='left', suffixes=(None, '_tipo'))

In [5]:
dataset['modification_date'] = pd.to_datetime(dataset['modification_date'], errors='coerce')
clean_dataset = dataset[(dataset["tipo"] == 1) & (dataset["estado"].isin([2,6])) & (dataset['modification_date'] < '2024-05-09')]

In [6]:
# Load from disk the text to translate dictionary
fields_to_translate = ["desc_problema", "problema", "descripcion"]
text_to_translate = {}
for text in fields_to_translate:
    text_to_translate[text] = pd.read_csv(f"../DATA/{text}.csv", sep='¬', encoding='utf-8-sig')

In [7]:
desc_problema_translated = pd.read_csv("../DATA/desc_problema_translated.csv", sep='¬', encoding='utf-8-sig', engine='python')
descripcion_translated = pd.read_csv("../DATA/descripcion_translated.csv", sep='¬', encoding='utf-8-sig', engine='python')
problema_translated = pd.read_csv("../DATA/problema_translated.csv", sep='¬', encoding='utf-8-sig', engine='python')# Data preprocessing (Merging the translated text)

In [8]:
# Delete rows with values (desc_problema, desc_problema_translated)
desc_problema_translated = desc_problema_translated[~desc_problema_translated["desc_problema_translated"].isin(["desc_problema_translated"])]
descripcion_translated = descripcion_translated[~descripcion_translated["descripcion_translated"].isin(["descripcion_translated"])]
problema_translated = problema_translated[~problema_translated["problema_translated"].isin(["problema_translated"])]

In [9]:
desc_problema_translated.count()

In [10]:
# Merge the translated text with the text_to_translate dataframe
desc_problema_translated = text_to_translate["desc_problema"].merge(desc_problema_translated, left_on="desc_problema", right_on="desc_problema", how="left")
descripcion_translated = text_to_translate["descripcion"].merge(descripcion_translated, left_on="descripcion", right_on="descripcion", how="left")
problema_translated = text_to_translate["problema"].merge(problema_translated, left_on="problema", right_on="problema", how="left")

In [11]:
# Fill NA with the original texts
desc_problema_translated.fillna({"desc_problema_translated": desc_problema_translated["desc_problema"]}, inplace=True)
descripcion_translated.fillna({"descripcion_translated": descripcion_translated["descripcion"]}, inplace=True)
problema_translated.fillna({"problema_translated": problema_translated["problema"]}, inplace=True)

In [12]:
desc_problema_translated.head(5)

In [13]:
# Merge the translated text with the original dataset
clean_dataset = clean_dataset.merge(desc_problema_translated, left_on="desc_problema", right_on="desc_problema", how="left")
clean_dataset = clean_dataset.merge(descripcion_translated, left_on="descripcion", right_on="descripcion", how="left")
clean_dataset = clean_dataset.merge(problema_translated, left_on="problema", right_on="problema", how="left")

In [14]:
clean_dataset.describe(include='all')

In [15]:
for column in clean_dataset.columns:
    print(f"Column: {column}")

In [16]:
# Get only the columns with the fields of interest
text_to_analyse = clean_dataset[['codigo','id_pieza','desc_problema_translated','descripcion_translated','problema_translated','cod_articulo']]
# Fill NA with empty string
text_to_analyse.fillna("", inplace=True)

In [17]:
text_to_analyse.loc[:, 'text_to_analyse'] = text_to_analyse['desc_problema_translated'] + ' ' + text_to_analyse['descripcion_translated'] + ' ' + text_to_analyse['problema_translated'] + ' ' + text_to_analyse['cod_articulo']

In [18]:
text_to_analyse.head(10)

## Add CODART from A3ERP

In [19]:
import pyodbc
# Def function to connect with sql server using pyodbc and query data
def query_data_a3(query,database):
    r"""
    Function to query data from a SQL database
    :param query: 
    :param database: 
    :return: pd.DataFrame
    
        Connection parameters:
        - user: voliveira
        - password: (get password from environment variable SQL_PASSWORD)
        - host: ROMPETECHOS\REPLICA
        - port: 53373
    """
    # Create the connection string
    user = 'voliveira'
    password = os.environ.get('SQL_PASSWORD')
    host = r'ROMPETECHOS'
    instance = 'REPLICA'
    port = '53373'
    conn_str = f"DRIVER=SQL Server;SERVER={host}\\{instance},{port};DATABASE={database};UID={user};PWD={password}"

    # Create the connection
    try:
        conn = pyodbc.connect(conn_str)
    except Exception as e:
        print(f'Error creating connection: {e}')
        return None
    
    # query the data
    try:
        data = pd.read_sql(query, conn)
    except Exception as e:
        print(f'Error: {e}')
        data = None
        
    return data

In [20]:
# Query the data from A3ERP
articulos = query_data_a3(f'SELECT CODART, DESCART, CAR1, CAR2, CAR3, CAR4 FROM dbo.ARTICULO','Altra')
caracteristicas = query_data_a3(f'SELECT * FROM dbo.CARACTERISTICAS;','Altra')

In [21]:
articulos.sample(10)

In [22]:
caracteristicas.sample(10)

In [23]:
def get_unique_caracteristicas(numcar):
    return caracteristicas[(caracteristicas['NUMCAR'] == numcar) & (caracteristicas['TIPCAR'] == 'A')][['CODCAR', 'DESCCAR']]



# Merging the characteristics with the articles
articulos = articulos\
    .merge(get_unique_caracteristicas(1), left_on='CAR1', right_on='CODCAR', how='left', suffixes=(None, '1'))
articulos = articulos\
    .merge(get_unique_caracteristicas(2), left_on='CAR2', right_on='CODCAR', how='left', suffixes=(None, '2'))
articulos = articulos\
    .merge(get_unique_caracteristicas(3), left_on='CAR3', right_on='CODCAR', how='left', suffixes=(None, '3'))
articulos = articulos\
    .merge(get_unique_caracteristicas(4), left_on='CAR4', right_on='CODCAR', how='left', suffixes=(None, '4'))

# Clean usuless columns
articulos = articulos.drop(['CODCAR', 'CODCAR2', 'CODCAR3', 'CODCAR4'], axis=1)
# Rename to match patterns
articulos = articulos.rename(columns={'DESCCAR': 'DESCCAR1'})

In [24]:
articulos.sample(10)

In [25]:
from thefuzz import process, fuzz

def find_best_match(cod, codart_list, return_score=True):
    best_match, score = process.extractOne(cod, codart_list, scorer=fuzz.token_set_ratio)
    return (best_match, score) if return_score else best_match

In [26]:
"""# Add the CODART to the text_to_analyse dataframe
text_to_analyse[['CODART_A3','Fuzzy_Score']] = (text_to_analyse['cod_articulo']
                                                .apply(lambda x: pd.Series(find_best_match(x, articulos['CODART'].values))))

# Save to disk
text_to_analyse[['cod_articulo','CODART_A3','Fuzzy_Score']].to_csv("../DATA/fuzzy_matches_w_scores.csv", sep='¬', encoding='utf-8-sig', index=False)"""

# Load fuzzy matches from disk
fuzzy_matches = pd.read_csv("../DATA/fuzzy_matches_w_scores.csv", sep='¬', encoding='utf-8-sig')
fuzzy_matches.drop_duplicates(inplace=True)

# Merge the fuzzy matches with the text_to_analyse dataframe
text_to_analyse = text_to_analyse.merge(fuzzy_matches, left_on='cod_articulo', right_on='cod_articulo', how='left')

In [27]:
text_to_analyse.sample(10)

In [28]:
# Count the number of NaN values in column CODART_A3
text_to_analyse['CODART_A3'].isna().sum()

In [29]:
# Fill NA with 0
text_to_analyse.fillna('0', inplace=True)

In [30]:
# Clean the text_to_analyse dataframe
#text_to_analyse = text_to_analyse[text_to_analyse['CODART_A3'] != '0']
#text_to_analyse = text_to_analyse[text_to_analyse['CODART_A3'].notna()]
text_to_analyse = text_to_analyse[text_to_analyse['text_to_analyse'].str.len() > 25]
# Clean rows with only NO FUNCIONA in text_to_analyse
text_to_analyse = text_to_analyse[text_to_analyse['text_to_analyse'].str.replace('NO FUNCIONA', '').str.len() > 25]
text_to_analyse.head()

In [31]:
# Merge the text_to_analyse with the articulos dataframe
text_to_analyse = text_to_analyse.merge(articulos, left_on='CODART_A3', right_on='CODART', how='left')
text_to_analyse.head()

In [32]:
text_to_analyse = text_to_analyse.fillna('')
text_to_analyse = text_to_analyse.copy()
text_to_analyse['text_to_analyse'] = text_to_analyse['desc_problema_translated'] + \
                           ' ' + text_to_analyse['descripcion_translated'] + \
                           ' ' + text_to_analyse['problema_translated'] + \
                           ' ' + text_to_analyse['CODART_A3']

text_to_analyse.head(10)

In [33]:
# Save to disk
os.makedirs(data_base_path, exist_ok=True)
text_to_analyse.to_csv(f"{data_base_path}/text_to_analyse.csv", sep='¬', encoding='utf-8-sig', index=False)

In [33]:
# Load from disk
text_to_analyse = pd.read_csv(f"../DATA/processed/2024-05-13/text_to_analyse.csv", sep='¬', encoding='utf-8-sig')
text_to_analyse.head()

In [34]:
group_by_family = text_to_analyse.groupby('DESCCAR3').size().sort_values(ascending=False)
cummulative_percentage = group_by_family.cumsum() / group_by_family.sum() * 100

# Bars
plt.figure(figsize=(20, 8))
plt.bar(group_by_family.index, group_by_family.values, zorder=2)
plt.xticks(rotation=90)
plt.xlabel('Family')
plt.ylabel('Number of problems')
plt.title('Number of problems by family')
plt.grid(zorder=0)

# Cumulative percentage
plt.twinx()
plt.plot(cummulative_percentage, color='red', marker='o', zorder=2)
plt.ylabel('Cumulative percentage')
# plot the point labels
for i in range(15):
    plt.text(i, cummulative_percentage.values[i], f"{cummulative_percentage.values[i]:.2f}", ha='center', va='bottom', zorder=3)
plt.grid(False)

plt.show()

In [291]:
# Define the families to remove
special_families = ['REEMPLAZOS','PACKS']
meanless_families = ['DOCUMENTACION','EXPOSITORES','TOBERAS','MP_CONSUMIBLES', 'OBS_CENTRALIZADO', 'MP_ELECTRONICA', 
                     'MP_TERMINADOS/SEMI', 'MP_MECANICOS', 'OBS_REJILLA MOTORIZADA', 
                     'OBS_COMUNES', 'OBS_DISTRIBUIDO', 
                     'DOCUMENTACION', 'OBS_DIFUSOR SIN REGULACIÓN', 'EXPOSITORES', 
                     'OBS_OTROS', 'MP_SERVICIOS', 'MP_I+D', 'OBS_REJILLA SIN REGULACIÓN', 
                     'OBS_PANTALLAS GRAFICAS', 'OBS_IB PRO USA', 'Soporte técnico', 
                     'PUESTA EN MARCHA', 'MP_DIFUSION', 'OBS_ALARMAS TÉCNICAS', 
                     'MP_CHAPAS', 'MP_CABLE', 'OBS_CABLE', 'MP_MOTORES', 'MERCHANDISING', 
                     'OBS_ZONEPAD', 'OBS_COMUNICACIONES', 'OBS_DIFUSOR MOTORIZADO', 
                     'OBS_CAJA DE MEZCLA', 'OBS_ANTREE']

In [37]:
# Remove the meanless families
text_to_analyse_clean = text_to_analyse[~text_to_analyse['DESCCAR3'].isin(meanless_families)]
text_to_analyse_clean = text_to_analyse_clean[~text_to_analyse['DESCCAR3'].isin(special_families)]

In [40]:
# Clean low similariy scores
threshold = 85
text_to_analyse_clean = text_to_analyse_clean[text_to_analyse_clean['Fuzzy_Score'] >= threshold]

In [315]:
"""count_meanless_families = text_to_analyse[text_to_analyse['DESCCAR3'].isin(meanless_families)]['codigo'].count()
count_special_families = text_to_analyse[text_to_analyse['DESCCAR3'].isin(special_families)]['codigo'].count()

teste = text_to_analyse[~text_to_analyse['DESCCAR3'].isin(meanless_families)]
teste = teste[~teste['DESCCAR3'].isin(special_families)]

count_fuzzy = teste[teste['Fuzzy_Score'] < 85]['codigo'].count()

count_total = count_meanless_families + count_fuzzy


print(f"Total rows before cleaning: {text_to_analyse.shape[0]}")
print(f"Total rows after cleaning cod articulo: {text_to_analyse_clean.shape[0]+count_special_families}")
print(f"Total rows with bad cod articulo: {count_total}")
print(f"Percentage of rows with bad cod articulo: {count_total / (text_to_analyse.shape[0]-count_special_families) * 100:.2f}%")
print(f"Total rows after cleaning families: {text_to_analyse_clean.shape[0]}")
print(f"Total 'REEMPLAZOS'+'PACKS': {count_special_families}")
print(f"Percentage of 'REEMPLAZOS'+'PACKS': {count_special_families / text_to_analyse_clean.shape[0] * 100:.2f}%")"""

""""""
# Total rows before cleaning: 35732
# Total rows after cleaning cod articulo: 31269
# Total rows with bad cod articulo: 4463
# Percentage of rows with bad cod articulo: 14.82%
# Total rows after cleaning families: 25661
# Total 'REEMPLAZOS'+'PACKS': 5608
# Percentage of 'REEMPLAZOS'+'PACKS': 21.85%

In [46]:
## Save to disk
#os.makedirs(data_base_path, exist_ok=True)
#text_to_analyse_clean.to_csv(f"{data_base_path}/text_to_analyse_clean.csv", sep='¬', encoding='utf-8-sig', index=False)

In [255]:
# Load from disk
text_to_analyse_clean = pd.read_csv(f"{data_base_path}/text_to_analyse_clean.csv", sep='¬', encoding='utf-8-sig')

## Train DocVec and save the model

In [256]:
import spacy
nlp = spacy.load('es_core_news_sm')
# nlp = spacy.load('es_core_news_md')
# nlp = spacy.load('es_core_news_sm')
import multiprocessing
from collections import OrderedDict
import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [257]:
# Function to preprocess the text
def preprocess_text(docs):
    texts = [doc for doc in nlp.pipe(docs, disable=["ner", "parser"])] 
    processed_texts = []
    for doc in texts:
        tokens = [token.text.lower() for token in doc if not token.is_punct and not token.is_stop and not token.is_space]
        processed_texts.append(' '.join(tokens))
    return processed_texts

In [258]:
# Class Model with comments
class CommentedDoc2Vec(Doc2Vec):
    def __init__(self, comment="", **kwargs):
        super().__init__(**kwargs)
        self.comment = comment

In [50]:
# Load train corpus from disk
corpus = pd.read_csv('../DATA/processed/2024-05-13/corpus.csv')
corpus['processed_text'] = preprocess_text(corpus['text_to_analyse'])
corpus.head()

In [51]:
common_kwargs = dict(
    vector_size=200, 
    epochs=20, 
    min_count=2,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=5, 
    hs=0,
    seed=0
)

In [ ]:
# PV-DBOW plain
model = CommentedDoc2Vec(
    dm=0, 
    comment=f"PV-DBOW-"
            f"v_size {common_kwargs['vector_size']}-"
            f"epochs {common_kwargs['epochs']}-"
            f"hs {common_kwargs['hs']}-"
            f"sample {common_kwargs['sample']}-"
            f"negative {common_kwargs['negative']}-"
            f"min_count {common_kwargs['min_count']}",
    **common_kwargs
)

In [ ]:
# Create TaggedDocument objects
tagged_data = [TaggedDocument(words=doc.split(), tags=[i]) for i, doc in enumerate(corpus['processed_text'])]

In [ ]:
# Build the vocabulary
model.build_vocab(tagged_data)
print("Model: %s : vocabulary scanned & state initialized" % model.comment)

In [ ]:
# Train the model using the Corpus
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
print("%s training completed" % model.comment)

In [ ]:
# Save the model
os.makedirs(model_base_path, exist_ok=True)
model_name = f"{model_base_path}/{type(model).__name__}_{model.comment.replace(' ', '_')}.model"
model.save(model_name)
print(f"Model saved at {model_name}")

## Load the model and infer vectors

In [259]:
# Load the model
model_name = "../MODELS/2024-05-13/CommentedDoc2Vec_PV-DBOW-v_size_200-epochs_20-hs_0-sample_0-negative_5-min_count_2.model"
model = CommentedDoc2Vec.load(model_name)
print(f'Model {model} loaded')

In [260]:
# Infer vectors for the text_to_analyse
text_to_analyse_clean['processed_text'] = preprocess_text(text_to_analyse_clean['text_to_analyse'])
text_to_analyse_clean['vector'] = text_to_analyse_clean['processed_text'].apply(lambda x: model.infer_vector(x.split()))

In [261]:
text_to_analyse_clean.sample(10)

## Calculate the similarity between the texts

In [262]:
# Read list of errors
errors = pd.read_csv("../DATA/TablaTipoErrorPostventa.csv", sep=';', header=1)[['Código','CODCAR3','CODCAR2','DESCFAM','Motivo General']]
errors.columns = ['ID_ERROR','CODCAR3','CODCAR2','DESCFAM','DESCRIPCION'] # Rename columns
errors['CODCAR2'] = errors['CODCAR2'].str.replace('-','0').astype(int) # Clean CODCAR2

In [263]:
# Infer vector for errors
errors['description_processed'] = preprocess_text(errors['DESCRIPCION'])
errors['vector'] = errors['description_processed'] .apply(lambda x: model.infer_vector(x.split()))

In [264]:
errors

In [265]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_score(vector, vector_error):
    return cosine_similarity(vector.reshape(1, -1), vector_error.reshape(1, -1))[0][0]

def calculate_mean_cosine_score(vector, vector_error, n=5):
    if vector.size == 0 or vector_error.size == 0:
        return np.nan  # Return NaN if there's no vector to compare
    cosine_scores = []
    for i in range(n):
        cosine_scores.append(
            calculate_cosine_score(vector, vector_error)
        )
    return np.mean(cosine_scores)

In [266]:
# Calculate the cosine similarity between the text_to_analyse and the errors
for index, row in errors.iterrows():
    # Create a condition for filtering
    condition = (text_to_analyse_clean['CAR3'] == row['CODCAR3'])
    if row['CODCAR2']:
        condition &= (text_to_analyse_clean['CAR2'] == row['CODCAR2'])
    
    if not text_to_analyse_clean.loc[condition, 'vector'].empty:
        text_to_analyse_clean.loc[condition, f'cosine_similarity_{row["ID_ERROR"]}'] = text_to_analyse_clean.loc[condition, 'vector'].apply(lambda x: calculate_mean_cosine_score(x, row['vector']))
    
    print(f"Error {row['ID_ERROR']} calculated")

In [267]:
text_to_analyse_clean.sample(10)

In [269]:
# Save text_to_analyse to disk
#text_to_analyse_clean.to_csv(f"{data_base_path}/text_to_analyse_with_errors.csv", sep='¬', encoding='utf-8-sig', index=False)

In [4]:
# Load text_to_analyse from disk
text_to_analyse_clean = pd.read_csv(f"{data_base_path}/text_to_analyse_with_errors.csv", sep='¬', encoding='utf-8-sig')

In [5]:
cosine_columns = [col for col in text_to_analyse_clean.columns if 'cosine_similarity_' in col]
text_to_analyse_clean[cosine_columns] = text_to_analyse_clean[cosine_columns].fillna(0) # Fill NA with 0
text_to_analyse_clean.loc[:, 'highest_score'] = text_to_analyse_clean[cosine_columns].max(axis=1)
text_to_analyse_clean.loc[:, 'highest_score_error'] = text_to_analyse_clean[cosine_columns].idxmax(axis=1).apply(lambda x: x.split('_')[-1])

In [6]:
text_to_analyse_clean.head(10)

In [7]:
top10_per_error = text_to_analyse_clean[['codigo','text_to_analyse', 'highest_score', 'highest_score_error']] \
    .groupby('highest_score_error', group_keys=False) \
    .apply(lambda x: x.nlargest(10, 'highest_score')) \
    .reset_index(drop=True)

top10_per_error.head(500)

## Visualize the results

In [8]:
results = text_to_analyse_clean[['vector', 'highest_score_error']]
# Convert string representations of lists to actual numpy arrays
results['vector'] = results['vector'].apply(lambda x: np.fromstring(x.strip("[]"), sep=' ') if isinstance(x, str) else x)

In [9]:
# Expand each vector into its own column
expanded_vectors = results['vector'].apply(pd.Series)
expanded_vectors.columns = [f'vector_{i}' for i in range(expanded_vectors.shape[1])]
results = pd.concat([results, expanded_vectors], axis=1)

In [333]:
X = results.drop(['vector', 'highest_score_error'], axis=1)
y = pd.to_numeric(results['highest_score_error'], errors='coerce')

In [334]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, random_state=0)
X_tsne = tsne.fit_transform(X)

In [342]:
# Plot the results
plt.figure(figsize=(20, 10))
plt.scatter(X_tsne[:,0], X_tsne[:,1], c=y, cmap='tab20')
plt.colorbar()
plt.show()

In [343]:
# Create TSNE with 3 components
tsne = TSNE(n_components=3, random_state=0)
X_tsne_3d = tsne.fit_transform(X)

In [350]:
# Plot data in 3D
%matplotlib qt6

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(X_tsne_3d[:,0], X_tsne_3d[:,1], X_tsne_3d[:,2], c=y, cmap='tab20')
# Add legend
plt.legend(*scatter.legend_elements(num=10), title="Classes")
plt.show()